In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

In [7]:
data = pd.read_csv('features_3_sec.csv')
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [8]:
X = data.drop(['filename', 'length', 'label'], axis=1)
y = data['label']

In [9]:
#dividimos entre el entrenamieto y la prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=24)

In [ ]:
class Classificador(nn.Module):
    def __init__(self,input_size,hidden,output_size):
        '''
        Define las caracteristicas de una red completamente conectada 
        de tres capas, recibe la cantidad de elementos de entrada, el 
        número de capas ocultas y el número de elementos de salida. 
        Entre cada capa agrega una función de activación logistica.
        '''
        super(Classificador,self).__init__()
        
        #Vamos a considerar que las capas no tiene bias
        self.c1 = nn.Linear(in_features = input_size, out_features = hidden, bias = False)
        self.c2 = nn.Linear(in_features = hidden, out_features = output_size, bias = False)

        #funcion de error
        self.funcion_error = nn.CrossEntropyLoss()

        #error actual de la red
        self.error = 0
        
    def feed_forward(self,X):
        '''
        Define una función que de como resultado realizar la propagación
        hacia adelante de los elementos de X en la red definida.
        '''
        capa_oculta = torch.sigmoid(self.c1(X))
        salida = torch.sigmoid(self.c2(capa_oculta))
        # salida = self.c2(capa_oculta)

        return salida
    
    def back_propagate(self,X,Y):
        '''
        Define una función que realice la propagación hacia atras usando 
        la función de error de entropia cruzada.
        '''
        #hacemos feed forward
        salida = self.feed_forward(X)

        #usamos la funcion de error de entropia cruzada y calculamos el error
        self.error = self.funcion_error(salida, Y)

        #hacemos backpropagation
        self.error.backward()
        
    def train(self,train_X,train_Y,optimizer,ciclos=100):
        '''
        Define una función de entrenamiento para la red, la cual utilice
        al conjunto de entrenamiento y el algoritmo de optimización que se 
        obtenga como parametro. Al finalizar los ciclos muestra la gráfica 
        del error.
        '''
        #usamos el metodo de optimizacion dado
        self.optimizer = optimizer
        errores_lista = []
        epocas = [i for i in range(1, ciclos + 1)]

        #entrenamos la red
        for _ in range(ciclos):
            #"limipiamos los gradientes"
            self.optimizer.zero_grad()
            #hacemos backpropagation y actualizamos los pesos
            self.back_propagate(train_X, train_Y)
            self.optimizer.step()

            #calculamos el error y lo guardamos en la lista
            errores_lista.append(self.error.item())

        #graficamos el error
        plt.plot(epocas, errores_lista, color = '#8315C2')
        plt.grid(True)
        plt.xlabel('Cíclos')
        plt.ylabel('Error')
        plt.title('Error de la Red Neuronal')
        plt.show()        

    def confusion(self,test_X,test_Y):
        '''
        Muestra la matriz de confusión que presenta los valores actuales de
        la red, respecto al conjunto de datos que se decida usar.
        '''
        with torch.no_grad():
            #obtenemos las predicciones y redondeamos 
            y_pred =self.feed_forward(test_X)
            y_pred_class = y_pred.round()
            
            #Primera matriz de confusion
            matriz_confusion = torch.zeros((2, 2))

            TP = torch.sum(torch.logical_and(y_pred_class, test_Y))
            matriz_confusion[0, 0] = TP

            TN = torch.logical_and(torch.logical_not(y_pred_class), torch.logical_not(test_Y))
            matriz_confusion[1, 1] = torch.sum(TN)

            FP = torch.logical_and(y_pred_class, torch.logical_not(test_Y))
            matriz_confusion[1, 0] = torch.sum(FP)

            FN = torch.logical_and(torch.logical_not(y_pred_class), test_Y)
            matriz_confusion[0, 1] = torch.sum(FN)
            
            #pasamos la matriz a un data frame
            df = pd.DataFrame(matriz_confusion, index = ['Positivo', 'Negativo'], columns = ['Positivo', 'Negativo'])

            print(df)
            print()
            
            #segunda matriz de confusion
            pred = torch.max(self.feed_forward(test_X), 1)[1]
            ytrue = torch.max(test_Y, 1)[1]
            matriz_confusion = confusion_matrix(ytrue, pred)
            print(matriz_confusion)

            #calculamos el accuracy
            accuracy = (pred == ytrue).sum().item() / len(pred)
            print(f'Accuracy: {accuracy}')

In [ ]:
optimizer = torch.optim.Adam(RedAdam.parameters(), lr = 0.01)